## Data Cleanup and Combining

In [1]:
# Import Dependencies
import os
import pandas as pd
import numpy as np
from datetime import datetime

Data sets from citibike hold several million rows, so February, May, August and November will be used to represent each season

In [2]:
# Import data

feb_data = pd.read_csv('data/202102-citibike-tripdata.csv', dtype={'start_station_id': 'string', 'end_station_id': 'string'})
may_data = pd.read_csv('data/202105-citibike-tripdata.csv', dtype={'start_station_id': 'string', 'end_station_id': 'string'})
aug_data = pd.read_csv('data/202108-citibike-tripdata.csv', dtype={'start_station_id': 'string', 'end_station_id': 'string'})
nov_data = pd.read_csv('data/202111-citibike-tripdata.csv', dtype={'start_station_id': 'string', 'end_station_id': 'string'})

In [3]:
# Check if data types for each corresponding column are the same among all data frames
# Concatenate all of the data frames to compare data types
data_types_df = pd.concat([feb_data.dtypes, may_data.dtypes, aug_data.dtypes, nov_data.dtypes], \
                              axis=1)

# Rename columns to make the data frame easier to understand
data_types_df.columns = ['feb', 'may', 'aug', 'nov']

# Check for inconsistent data types in corresponding columns
data_types_df

,feb,may,aug,nov
ride_id,object,object,object,object
rideable_type,object,object,object,object
started_at,object,object,object,object
ended_at,object,object,object,object
start_station_name,object,object,object,object
start_station_id,string,string,string,string
end_station_name,object,object,object,object
end_station_id,string,string,string,string
start_lat,float64,float64,float64,float64
start_lng,float64,float64,float64,float64


In [4]:
combined_data = pd.concat([feb_data, may_data, aug_data, nov_data])
combined_data.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,BBA33D73DECE976F,docked_bike,2021-02-26 16:38:54,2021-02-26 16:44:37,E 84 St & Park Ave,7243.04,E 78 St & 2 Ave,7057.07,40.778626,-73.957720,40.772797,-73.955778,casual
1,B63D7AFF9AC5B6D4,docked_bike,2021-02-17 11:09:11,2021-02-17 11:26:47,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
2,52B829195C469D99,docked_bike,2021-02-26 18:33:29,2021-02-26 19:05:41,Macon St & Nostrand Ave,4214.03,Lefferts Pl & Franklin Ave,4222.02,40.680983,-73.950047,40.680342,-73.955769,casual
3,19C84ECA2B468476,docked_bike,2021-02-26 12:48:35,2021-02-26 13:07:24,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
4,C0DDB771E70D9DF5,docked_bike,2021-02-25 17:23:22,2021-02-25 17:28:20,Madison Ave & E 26 St,6131.12,W 37 St & 5 Ave,6398.06,40.742685,-73.986713,40.750380,-73.983390,member


In [5]:
#Check to make sure that the bike type descritions are consistent across each month
print(combined_data['rideable_type'].unique())

['docked_bike' 'electric_bike' 'classic_bike']


In [6]:
# Convert 'docked bike' to 'classic bike'
combined_data = combined_data.replace('docked_bike', 'classic_bike')
print(combined_data['rideable_type'].unique())

['classic_bike' 'electric_bike']


In [7]:
#Confirm that the data set is <15,000,000 so Tableau will be able to handle the data
combined_data['rideable_type'].count()

8606265

In [8]:
# Drop un-used columns
combined_data = combined_data.drop(columns='ride_id')
combined_data.head()

,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,classic_bike,2021-02-26 16:38:54,2021-02-26 16:44:37,E 84 St & Park Ave,7243.04,E 78 St & 2 Ave,7057.07,40.778626,-73.957720,40.772797,-73.955778,casual
1,classic_bike,2021-02-17 11:09:11,2021-02-17 11:26:47,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
2,classic_bike,2021-02-26 18:33:29,2021-02-26 19:05:41,Macon St & Nostrand Ave,4214.03,Lefferts Pl & Franklin Ave,4222.02,40.680983,-73.950047,40.680342,-73.955769,casual
3,classic_bike,2021-02-26 12:48:35,2021-02-26 13:07:24,Macon St & Nostrand Ave,4214.03,Bond St & Fulton St,4479.06,40.680983,-73.950047,40.689622,-73.983043,member
4,classic_bike,2021-02-25 17:23:22,2021-02-25 17:28:20,Madison Ave & E 26 St,6131.12,W 37 St & 5 Ave,6398.06,40.742685,-73.986713,40.750380,-73.983390,member


In [12]:
combined_data['start_station_name'].str.replace(' ', '')
combined_data['end_station_name'].str.replace(' ', '')

0                      E78St&2Ave
1                 BondSt&FultonSt
2          LeffertsPl&FranklinAve
3                 BondSt&FultonSt
4                      W37St&5Ave
                    ...          
2159279    AstoriaParkS&ShoreBlvd
2159280           LenoxAve&W126St
2159281                E51St&1Ave
2159282           LenoxAve&W126St
2159283               OldFultonSt
Name: end_station_name, Length: 8606265, dtype: object

In [13]:
# Export aggregated data to csv file
combined_data.to_csv('data/citi_bike_2021.csv', sep=',', index=False)

In [44]:
#Select random sample of 100,000 rows of data for working purposes
random_df = combined_data.sample(n=100000)

random_df.to_csv('data/random_sample.csv', sep=',', index=False)